In [2]:
import os
import time
import numpy as np
import pandas as pd
import sys
import mhd_utils_3d 
import mhd_utils_3d_short
import mhd_utils_3d_ushort
import matplotlib.pyplot as plt
from struct import *
from PIL import Image

import csv 
import pickle
import glob
import random

import theano
import theano.tensor as T

Using gpu device 0: GeForce GTX TITAN X


In [2]:
pkl_data = "/home/dl-box/Liver/sotsuron_data/Cyst106_NC_classificate.pkl"

listData=[]
intensity=0 
label=0 

train_listData_intensity=[]
train_listData_label=[]

valid_listData_intensity=[]
valid_listData_label=[]

test_listData_intensity=[]
test_listData_label=[]

↓数確認

In [4]:
path = "/home/dl-box/Liver/Cyst_data1102/liver_data"
image_array1, meta_header1 = mhd_utils_3d_short.load_raw_data_with_mhd(path+'/data_000_050/data050.mhd')
image_array2, meta_header2 = mhd_utils_3d_ushort.load_raw_data_with_mhd(path+'/LiverMask_000_050/LiverMask_Srr050_PV.mhd') 
image_array3, meta_header3 = mhd_utils_3d_ushort.load_raw_data_with_mhd(path+'/TumorMask_000_050/TumorMask_Srr050_PV.mhd')
data_indx=0
chose_i=[]
chose_l=[]
Data_intensity=[]
Data_label=[]

gutr_intensity=[]
gutr_label=[]
guval_intensity=[]
guval_label=[]

for z in range(0,image_array1.shape[0]):
    for y in range(0,image_array1.shape[1]):
        for x in range(0,image_array1.shape[2]):       
            if image_array2[z][y][x] == 1:
                if image_array3[z][y][x] == 1:
                    break
                else:    
                    data_indx += 1
                    new_img1 = ((image_array1[z,y-14:y+15,x-14:x+15]*1.0)/256)
                    intensity1 = new_img1.reshape((1,841))
                    label_L = 0
                    chose_i.append(intensity1)
                    chose_l.append(label_L)

In [5]:
data_indx

348892

In [51]:
data_indx = 0

chose_i=[]
chose_l=[]
Data_intensity=[]
Data_label=[]

kitr_intensity=[]
kitr_label=[]
kival_intensity=[]
kival_label=[]

for z in range(0,image_array1.shape[0]):
    for y in range(0,image_array1.shape[1]):
        for x in range(0,image_array1.shape[2]):       
            if image_array3[z][y][x] == 1:
                data_indx += 1
                new_img1 = ((image_array1[z,y-14:y+15,x-14:x+15]*1.0)/256)
                intensity1 = new_img1.reshape((1,841))
                label_T = 1
                chose_i.append(intensity1)
                chose_l.append(label_T)
 

In [52]:
data_indx

16814

---create the new dataset---

In [30]:
path = "/home/dl-box/Liver/Cyst_data1016"
image_array1, meta_header1 = mhd_utils_3d.load_raw_data_with_mhd(path+'/Cyst_data/data110.mhd')
image_array2, meta_header2 = mhd_utils_3d.load_raw_data_with_mhd(path+'/Cyst_PV_Liver/LiverMask_Srr110_PV.mhd') 
image_array3, meta_header3 = mhd_utils_3d.load_raw_data_with_mhd(path+'/Cyst_PV_Tumor/TumorMask_Srr110_PV.mhd')
data_indx=0
chose_i=[]
chose_l=[]
Data_intensity=[]
Data_label=[]

gutr_intensity=[]
gutr_label=[]
guval_intensity=[]
guval_label=[]

for z in range(0,image_array1.shape[0]):
    for y in range(0,image_array1.shape[1]):
        for x in range(0,image_array1.shape[2]):       
            if image_array2[z][y][x] == 1:
                if image_array3[z][y][x] == 1:
                    break
                else:    
                    data_indx += 1
                    new_img1 = ((image_array1[z,y-7:y+8,x-7:x+8]*1.0)/256)
                    intensity1 = new_img1.reshape((1,225))
                    label_L = 0
                    #print label_v
                    chose_i.append(intensity1)
                    chose_l.append(label_L)

new_i=random.sample(chose_i, 2850)
new_l=random.sample(chose_l, 2850)

for i in range(0,2850):
    Data_intensity.append(new_i[i])
    Data_label.append(new_l[i])
    
random.shuffle(Data_intensity)

for i in range(0,2850):
    if i%5 != 0:
        gutr_intensity.append(Data_intensity[i])
        gutr_label.append(Data_label[i])
    else:
        guval_intensity.append(Data_intensity[i])
        guval_label.append(Data_label[i])

In [31]:
data_indx = 0

chose_i=[]
chose_l=[]
Data_intensity=[]
Data_label=[]

kitr_intensity=[]
kitr_label=[]
kival_intensity=[]
kival_label=[]

for z in range(0,image_array1.shape[0]):
    for y in range(0,image_array1.shape[1]):
        for x in range(0,image_array1.shape[2]):       
            if image_array3[z][y][x] == 1:
                data_indx += 1
                new_img1 = ((image_array1[z,y-7:y+8,x-7:x+8]*1.0)/256)
                intensity1 = new_img1.reshape((1,225))
                label_T = 1
                chose_i.append(intensity1)
                chose_l.append(label_T)
                
new_i=random.sample(chose_i, 2850)
new_l=random.sample(chose_l, 2850)
for i in range(0,2850):
    Data_intensity.append(new_i[i])
    Data_label.append(new_l[i])
               
random.shuffle(Data_intensity)

for i in range(0,2850):
    if i%5 != 0:
        kitr_intensity.append(Data_intensity[i])
        kitr_label.append(Data_label[i])
    else:
        kival_intensity.append(Data_intensity[i])
        kival_label.append(Data_label[i])

In [32]:
for i in range(0,4560):
    if i%2 != 0:
        train_listData_intensity.append(gutr_intensity[i/2])
        train_listData_label.append(gutr_label[i/2])
    else:
        train_listData_intensity.append(kitr_intensity[i/2])
        train_listData_label.append(kitr_label[i/2])

for i in range(0,1140):
    if i%2 != 0 :
        valid_listData_intensity.append(guval_intensity[i/2])
        valid_listData_label.append(guval_label[i/2])
    else:
        valid_listData_intensity.append(kival_intensity[i/2])
        valid_listData_label.append(kival_label[i/2])

In [33]:
valid_listData=[]
test_listData=[]
train_listData=[]
listData=[]

valid_listData.append(valid_listData_intensity)
valid_listData.append(valid_listData_label)

train_listData.append(train_listData_intensity)
train_listData.append(train_listData_label)

listData.append(train_listData)
listData.append(valid_listData)

pickle.dump(listData, open(pkl_data,"w"))

---create test dataset---

In [10]:
import csv
listData=[]
pkl_data = "/home/dl-box/Liver/sotsuron_data/test_data/Meta504_NC_classificate.pkl"
path = "/home/dl-box/Liver/sotsuron_data/META/NC"

f = open(path+'/Meta504_NC.csv','ab')
Write = csv.writer(f)

new_img1 = []
listData=[]
intensity=0 
label=0 

test_listData_intensity=[]
test_listData_label=[]

image_array1, meta_header1 = mhd_utils_3d.load_raw_data_with_mhd(path+'/META504_NC.mhd')
image_array2, meta_header2 = mhd_utils_3d.load_raw_data_with_mhd(path+'/LiverMask_Srr504_NC.mhd') 
image_array3, meta_header3 = mhd_utils_3d.load_raw_data_with_mhd(path+'/TumorMask_Srr504_NC.mhd')
data_indx = 0
for z in range(0,image_array1.shape[0]):
    for y in range(0,image_array1.shape[1]):
        for x in range(0,image_array1.shape[2]):       
            #if image_array2[z][y][x] == 1:
            if image_array3[z][y][x] == 1:

                data_indx += 1
                new_img1 = ((image_array1[z,y-14:y+15,x-14:x+15]*1.0)/256)
                
                new_img3 = ((sum(image_array3[z,y-14:y+15,x-14:x+15])))
                new_img3 = sum(new_img3)
                if new_img3 > 336:
                #rint new_img1.shape
                    intensity1 = new_img1.reshape((1,841))
                    label_T= 0
                    test_listData_intensity.append(intensity1)
                    test_listData_label.append(label_T)

                    listData.append(np.array([x,y,z]))

for i in range(len(listData)):
    Write.writerow((listData[i]))
f.close()    


In [60]:
listData=[]
f = open(path+'/Cyst_data1102/liver_data/output_029.csv','ab')
Write = csv.writer(f)

for z in range(0,image_array1.shape[0]):
    for y in range(0,image_array1.shape[1]):
        for x in range(0,image_array1.shape[2]):       
            if image_array3[z][y][x] == 1:
                data_indx += 1
                new_img1 = ((image_array1[z,y-14:y+15,x-14:x+15]*1.0)/256)
                intensity1 = new_img1.reshape((1,841))
                label_T = 1
                test_listData_intensity.append(intensity1)
                test_listData_label.append(label_T)
                
                listData.append(np.array([x,y,z]))

for i in range(len(listData)):
    Write.writerow((listData[i]))
f.close()   

In [11]:
test_listData=[]
listData=[]

test_listData.append(test_listData_intensity)
test_listData.append(test_listData_label)

listData.append(test_listData)

pickle.dump(listData, open(pkl_data,"w"))